In [1]:
import cv2
import numpy as np
import time

In [2]:
# 初始化摄像头
cap = cv2.VideoCapture(4)

In [3]:
# 设置摄像头的帧大小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 980)

True

In [ ]:
#lower_blue = np.array([53, -50, -50])    (53, 100, -54, 2, -45, -2)
#upper_blue = np.array([100, 2, -2])
lower_blue = np.array([100, 50, 50])    
upper_blue = np.array([130, 255, 255])
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

In [ ]:
start_time = time.time()

# 初始化点的坐标
#L1, L2, R1, R2 = (0, 0), (0, 0), (0, 0), (0, 0)
# 初始化点的坐标和名字
points = {'L1': (0, 0), 'L2': (0, 0), 'R1': (0, 0), 'R2': (0, 0)}

while True:
    # 读取一帧图像
    ret, frame = cap.read()

    if not ret:
        break

    # 转换图像到HSV色彩空间
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 使用颜色阈值找到蓝色目标
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    _, blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 寻找蓝色目标的区域
    for contour in blue_contours:
        if cv2.contourArea(contour) > 1000:
            # 获取目标的上下两个点
            x, y, w, h = cv2.boundingRect(contour)

            # 根据横坐标判断是左侧还是右侧
            if x < frame.shape[1] // 2:  # 左侧
                if points['L1'] == (0, 0):
                    points['L1'] = (x + w // 2, y)
                else:
                    points['L2'] = (x + w // 2, y + h)
            else:  # 右侧
                if points['R1'] == (0, 0):
                    points['R1'] = (x + w // 2, y)
                else:
                    points['R2'] = (x + w // 2, y + h)

    # 在图像中标注出四个点
    for name, point in points.items():
        cv2.circle(frame, point, 5, (0, 255, 0) if name.startswith('L') else (0, 0, 255), -1)  # 根据名字用绿色或红色标注
        cv2.putText(frame, f"{name}: {point}", (point[0] + 10, point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 255, 255), 2)

    # 在图像上显示帧速率
    fps = 1 / (time.time() - start_time)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 显示结果图像
    cv2.imshow('Frame', frame)

    # 检测按键，如果按下 'q' 键，则退出循环
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 更新起始时间
    start_time = time.time()

# 释放摄像头资源
cap.release()

# 关闭所有窗口
cv2.destroyAllWindows()